# naturerepublic

In [1]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
import json
import platform
import re
import time
import copy
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def getNumber(string):
    numExtracter = re.compile('[0-9]+')
    return int(''.join(numExtracter.findall(string)))
    

In [3]:
def getCategoryList(url_products):
    driver.get(url_products)
    html = driver.page_source
    soup = bs(html,'html.parser')
    categories = soup.find('ul',{'class':'tab_category'})
    categoryList = []
    for category in categories:
        try:
            categoryList.append(url_home + category.li.a['href'])
        except:
            pass

    return categoryList

In [11]:
def clickSeeMoreButton():
    while True:
        try:
            seeMoreButton = driver.find_element_by_class_name('btn_more')            
            seeMoreButton.click()
            time.sleep(5)
            
        except:
            break

In [5]:
def getItemList():
    html = driver.page_source
    soup = bs(html,'html.parser')
    items = soup.find('ul',{'product_list contents_list'})
    itemList = []
    for i, item in enumerate(items):
        try:
            itemList.append(url_home + item.a['href'])
        except:
            pass
    return itemList

In [18]:
def getItem():
    
    html = driver.page_source
    soup = bs(html, 'html.parser')

    name = soup.find('div',{'class':'sub_tit_wrap tit_pro_view'}).h2.get_text().strip()

    imageList = soup.find('div',{'class':'thumb_img'})
    images = [image['rel'] for image in imageList.ul.find_all('img')]

    categoryTexts = soup.find('div',{'class':'line-nav'}).find_all('a')
    category = ''
    for categoryText in categoryTexts:
        category += categoryText.get_text()+' > '
    category = category[:-3]

    volume = soup.find('dl',{'class':'volume'}).dd.get_text()

    price = soup.find('dl',{'class':'wrap-price-area money'}).find_all('dd')
    originalPrice = getNumber(price[0].get_text())
    if len(price)>1:
        salePrice = getNumber(price[1].get_text())
    else:
        salePrice = originalPrice

    brand = '네이처리퍼블릭'

    url = driver.current_url


    item={}
    item['name']=name
    item['image']=images
    item['category']=category
    item['volume']=volume
    item['originalPrice']=originalPrice
    item['salePrice']=salePrice
    item['brand']=brand
    item['url']=url

    items = []
    colors = soup.find('ul',{'class':'txt_img'})
    if colors:
        colors = [color.get_text().strip() for color in colors.find_all('li')]
        for color in colors:
            item_copy = copy.deepcopy(item)
            item_copy['color'] = color
            items.append(item_copy)
    else:
        items.append(item)
    display(items)
    return items

In [7]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)


In [25]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

url_home = 'https://www.naturerepublic.com'
url_products = 'https://www.naturerepublic.com/shop/goods_list.php?cid=000000000000000&depth=0'
driver.get(url_products)

In [26]:
start_time = time.time() 
#-------------------------------------------------------------#
categoryList = getCategoryList(url_products)
result = []
for category in categoryList:
    driver.get(category)
    driver.execute_script("window.scrollTo(0, 0)") 
    clickSeeMoreButton()
    itemList = getItemList()
    
    totalItemNum = driver.find_element_by_class_name('sel_cat').text
    if getNumber(totalItemNum) != len(itemList):
        print('상품 개수:',totalItemNum)
        print('크롤링된 개수:',itemList)
        print('더보기 버튼이 충분히 눌려지지 않았습니다.')
        raise 
        
    for item in itemList:
        driver.get(item)
        result += getItem()
#-------------------------------------------------------------#
print("--- %0.2f seconds ---" %(time.time() - start_time))

더보기 버튼 click 횟수 : 1
더보기 버튼 click 횟수 : 2
더보기 버튼 click 횟수 : 3
더보기 버튼 click 횟수 : 4
더보기 버튼 click 횟수 : 5
더보기 버튼 click 횟수 : 6
더보기 버튼 click 횟수 : 7
[{'name': '*행사* 히아 인텐스 로즈 앰플 세럼(+마스크 5매 증정)', 'image': ['http://lfyaklyqypvr852590.gcdn.ntruss.com/data/nature_data/images/product/00/00/01/21/17/b_0000012117.gif', 'http://lfyaklyqypvr852590.gcdn.ntruss.com/data/nature_data/images/addimg/00/00/01/21/17/b_00030030_add.gif', 'http://lfyaklyqypvr852590.gcdn.ntruss.com/data/nature_data/images/addimg/00/00/01/21/17/b_00030031_add.gif'], 'category': 'HOME > 카테고리 > 스킨케어', 'volume': '상세페이지 참조', 'originalPrice': 38000, 'salePrice': 26600, 'brand': '네이처리퍼블릭', 'url': 'https://www.naturerepublic.com/shop/goods_view.php?id=0000012117'}]
[{'name': '*행사* 히아 인텐스 로즈 크림 기획세트(+마스크 10매 증정)', 'image': ['http://lfyaklyqypvr852590.gcdn.ntruss.com/data/nature_data/images/product/00/00/01/21/16/b_0000012116.gif', 'http://lfyaklyqypvr852590.gcdn.ntruss.com/data/nature_data/images/addimg/00/00/01/21/16/b_00030028_add.gif', 

TypeError: object of type 'NoneType' has no len()

In [21]:
output = json.dumps(result,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'naturerepublic.json')